In [ ]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_percentage_error

from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, classification_report


from snowflake.connector import connect
session = get_active_session()

## Revenue Forecasting

In [ ]:
df = session.sql("""
    SELECT *
    FROM BRAZILIAN_ECOM_SCHEMA.REVENUE_FORECAST_FEATURES
    ORDER BY ORDER_DATE
""").to_pandas()

train = df[df["ORDER_DATE"] < "2018-01-01"]
test  = df[df["ORDER_DATE"] >= "2018-01-01"]

X_train = train.drop(columns=["ORDER_DATE", "TOTAL_REVENUE"])
y_train = train["TOTAL_REVENUE"]

X_test = test.drop(columns=["ORDER_DATE", "TOTAL_REVENUE"])
y_test = test["TOTAL_REVENUE"]

model = XGBRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

model.fit(X_train, y_train)

preds = model.predict(X_test)

mape = mean_absolute_percentage_error(y_test, preds)
print(f"MAPE: {mape:.4f}")

In [ ]:

CREATE TABLE DBT_DB.BRAZILIAN_ECOM_SCHEMA.REVENUE_FORECAST_PREDICTIONS (
    FORECAST_DATE DATE NOT NULL,
    PREDICTED_REVENUE NUMBER(18,2) NOT NULL,
    MODEL_VERSION STRING NOT NULL,
    CREATED_AT TIMESTAMP_NTZ NOT NULL
);

In [ ]:
pred_df = (
    pd.DataFrame({
        "FORECAST_DATE": test["ORDER_DATE"].dt.date,
        "PREDICTED_REVENUE": preds.astype(float),
        "MODEL_VERSION": "xgb_v1",
        "CREATED_AT": pd.Timestamp.now().to_pydatetime()
    })
    .reset_index(drop=True)
)

session.write_pandas(
    pred_df,
    table_name="REVENUE_FORECAST_PREDICTIONS",
    schema="BRAZILIAN_ECOM_SCHEMA",
    database="DBT_DB",
    auto_create_table=False,
    overwrite=False
)

## Order Forecasting

In [ ]:
df = session.sql("""
    SELECT *
    FROM BRAZILIAN_ECOM_SCHEMA.ORDER_FORECAST_FEATURES
    ORDER BY ORDER_DATE
""").to_pandas()

df.dropna(inplace=True) #Drops the first 6 rows (for 7 day moving average)

train = df[df["ORDER_DATE"] < "2018-01-01"]
test  = df[df["ORDER_DATE"] >= "2018-01-01"]

X_train = train.drop(columns=["ORDER_DATE", "TOTAL_ORDERS"])
y_train = train["TOTAL_ORDERS"]

X_test = test.drop(columns=["ORDER_DATE", "TOTAL_ORDERS"])
y_test = test["TOTAL_ORDERS"]

model = XGBRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

model.fit(X_train, y_train)

preds = model.predict(X_test)

mape = mean_absolute_percentage_error(y_test, preds)
print(f"MAPE: {mape:.4f}")

In [ ]:
CREATE TABLE if not exists DBT_DB.BRAZILIAN_ECOM_SCHEMA.ORDER_FORECAST_PREDICTIONS (
    FORECAST_DATE DATE NOT NULL,
    PREDICTED_ORDER NUMBER(18,2) NOT NULL
);

In [ ]:
pred_df = (
    pd.DataFrame({
        "FORECAST_DATE": test["ORDER_DATE"].dt.date,
        "PREDICTED_ORDER": preds.astype(float)
    })
    .reset_index(drop=True)
)

session.write_pandas(
    pred_df,
    table_name="ORDER_FORECAST_PREDICTIONS",
    schema="BRAZILIAN_ECOM_SCHEMA",
    database="DBT_DB",
    auto_create_table=False,
    overwrite=False
)

## Churn Classification

In [ ]:
df = session.sql("""
    SELECT *
    FROM BRAZILIAN_ECOM_SCHEMA.CUSTOMER_CHURN_DATASET
""").to_pandas()

X = df.drop(columns=["CUSTOMER_ID","IS_CHURNED"])
y = df["IS_CHURNED"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

model = XGBClassifier(
    n_estimators=300,
    max_depth=3,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric="logloss",
    random_state=42
)

model.fit(X_train, y_train)

probs = model.predict_proba(X_test)[:, 1]
preds = (probs > 0.5).astype(int)

print("ROC AUC:", roc_auc_score(y_test, probs))
print(classification_report(y_test, preds))

In [ ]:
importance = pd.DataFrame({
    "feature": X_train.columns,
    "importance": model.feature_importances_
}).sort_values("importance", ascending=False)

importance

In [ ]:
CREATE TABLE IF NOT EXISTS DBT_DB.BRAZILIAN_ECOM_SCHEMA.CUSTOMER_CHURN_PREDICTIONS (
    customer_id STRING,
    churn_probability FLOAT,
    churn_prediction INTEGER
);

In [ ]:
pred_df = pd.DataFrame({
    "CUSTOMER_ID": df["CUSTOMER_ID"],
    "CHURN_PROBABILITY": model.predict_proba(X)[:, 1],
    "CHURN_PREDICTION": (model.predict_proba(X)[:, 1] > 0.5).astype(int)
}).reset_index(drop=True)

session.write_pandas(
    pred_df,
    table_name="CUSTOMER_CHURN_PREDICTIONS",
    schema="BRAZILIAN_ECOM_SCHEMA",
    database="DBT_DB",
    auto_create_table=False,
    overwrite=False
)
